In [1]:
debug = False

___
___
# **Librairies**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import VotingRegressor

import xgboost as xgb
import lightgbm as lgb
import catboost
from lightgbm.sklearn import LGBMClassifier
from catboost import CatBoostClassifier
from tqdm import tqdm
import time

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

___
___
# **Open Data**

In [3]:
df       = pd.read_csv('/kaggle/input/trustii-aim/train.csv')
test_df  = pd.read_csv('/kaggle/input/trustii-aim/test.csv')

print(df.shape, test_df.shape)
display(df.head())

(22671, 13) (9717, 13)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite
0,SAVEURS ASIATIQUES,50846842800034,RUE DES FRERES LUMIERE,33130,Bègles,23-110093-1,2024-02-05T01:00:00+01:00,Restaurant,Satisfaisant,NaN,44.797031_-0.535231,Restaurant,Autres
1,EPICERIE ROND POINT,90472398800013,48 AV DU ROND POINT,93250,Villemomble,23-056603-1,2023-07-17T02:00:00+02:00,Libre service|Alimentation générale,Satisfaisant,NaN,48.884745_2.499984,Libre service|Alimentation générale,Autres
2,AUBERGE DES OLIVIERS,48792443300013,ROUTE DE LA CANONICA,20290,Lucciana,23-018114-1,2023-03-08T01:00:00+01:00,Restaurant,Très satisfaisant,NaN,42.541715_9.46286,Restaurant,Autres
3,LES HUITRES DES FLOTS,80481822700022,RUE DU PONT DES BERNES,50550,Saint-Vaast-la-Hougue,23-011747-1,2023-02-15T01:00:00+01:00,Purification/Expédition de coquillages,Satisfaisant,50562016,49.592002_-1.28678,NaN,Produits de la mer et d'eau douce
4,COLONNA DOMINIQUE-ANTOINE,53444598600019,Partinello,20147,Partinello,23-097885-1,2023-11-21T01:00:00+01:00,Producteur fermier,Très satisfaisant,NaN,42.306077_8.67833,Producteur fermier,Autres


In [4]:
if debug :
    print("DEBUG IS ON ! ")
    df = df.sample(n = 500, random_state=12).reset_index(drop=True)
    test_df = test_df.sample(n = 100, random_state=12).reset_index(drop=True)
    
print(df.shape)

(22671, 13)


In [5]:
dico_y = {'A corriger de manière urgente' : 0,
          'A améliorer' : 1,
          'Satisfaisant' : 2,
          'Très satisfaisant' : 3,
         }
dico_y_INVERSE = {v:k for k, v in dico_y.items()}

In [6]:
Act = list(set(list(df['APP_Libelle_activite_etablissement'].unique()) + list(df['filtre'].unique())))
Activites = set()
for act in Act :
    if str(act).lower() in ('none', 'nan') :
        continue
    for x in act.split('|') :
        if len(x)>1 :
            Activites.add(x)
        
Activites = sorted(Activites)
Activites[:10]

["Abattage de produits de l'aquaculture",
 'Alimentation générale',
 'Autres activités de remise directe',
 'Boucherie-Charcuterie',
 'Boulangerie-Pâtisserie',
 "Caves d'affinage",
 "Centre d'emballage des oeufs",
 "Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage",
 "Chaîne d'abattage de volaille/lagomorphe/petit gibier",
 'Chocolatier']

In [7]:
# Copie des colonnes
for k in ['APP_Libelle_activite_etablissement', 'Libelle_commune', 'APP_Libelle_etablissement', 'Adresse_2_UA'] :
    df[f"{k}_initial"] = df[k].copy()
    test_df[f"{k}_initial"] = test_df[k].copy()

___
___
# **Add SIRET info**

In [8]:
info_SIRET = pd.read_csv('/kaggle/input/trustii-siret/info_SIRET.csv')

info_SIRET.drop(columns = ['SIRET', 'Date_inspection'], inplace=True)

print(info_SIRET.shape)
info_SIRET.head(3)

(32388, 21)


,Numero_inspection,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF
0,16218678,10,3,2,NaN,1,5,0,26,2,1,25,NaN,NaN,-1,-1,-1,-1,6,0,1
1,16728080,0,2,0,2.0,1,5,0,10,0,0,25,2054.0,2071.0,2017,7,2017,6,1,1,1
2,16728383,10,0,0,2.0,1,5,0,1,1,0,25,-269.0,683.0,2023,11,2021,4,0,2,1


In [9]:
# Pour que la jointure fonctionne, changement de type des dates
#df['date_str'] = df['Date_inspection'].astype(str).apply(lambda x : x[:10])
#test_df['date_str'] = test_df['Date_inspection'].astype(str).apply(lambda x : x[:10])
#info_SIRET['date_str'] = info_SIRET['Date_inspection'].astype(str).apply(lambda x : x[:10])
#info_SIRET.drop(columns = ['Date_inspection'], inplace=True)

In [10]:
# Jointure
df = pd.merge(df,
              info_SIRET,
              how='left',
              on=['Numero_inspection'],
             )

# Show
print(df.shape)
df.head(1)

(22671, 37)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF
0,SAVEURS ASIATIQUES,50846842800034,RUE DES FRERES LUMIERE,33130,Bègles,23-110093-1,2024-02-05T01:00:00+01:00,Restaurant,Satisfaisant,NaN,44.797031_-0.535231,Restaurant,Autres,Restaurant,Bègles,SAVEURS ASIATIQUES,RUE DES FRERES LUMIERE,1,3,0,2.0,1,5,0,0,0,0,10,4536.0,4813.0,2011,9,2010,12,0,0,0


In [11]:
test_df = pd.merge(test_df,
                  info_SIRET,
                  how='left',
                  on=['Numero_inspection'],
                 )

# Show
print(test_df.shape)
test_df.head(1)

(9717, 37)


,trustii_id,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF
0,1,LE GALL GEORGES,31917109600070,LIEU DIT MANIFAIX,63700,Lapeyrouse,23-003935-1,2023-08-08T02:00:00+02:00,Producteur fermier,NaN,46.224095_2.93491,Producteur fermier,Autres,Producteur fermier,Lapeyrouse,LE GALL GEORGES,LIEU DIT MANIFAIX,10,3,0,1.0,1,5,0,12,1,0,25,1533.0,1533.0,2019,5,2019,5,3,0,0


In [12]:
#df.drop(columns = ['date_str'], inplace=True)
#test_df.drop(columns = ['date_str'], inplace=True)

___
___
# **TFIDF**

In [13]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('french') + stopwords.words('english')
#stopwords_list = [x.upper() for x in stopwords_list]
print(len(stopwords_list))
stopwords_list[:10]

336


['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle']

In [14]:
# Fonction pour supprimer les nombres d'une chaine de caractères
def suppression_nombres(txt):
    ''' Supprime les nombres dans une chaine de caractères '''
    # Suppression des chiffres
    for chiffre in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        txt = txt.replace(chiffre, '')
    # Suppression des espaces en trop
    txt = re.sub(r'\s+', ' ', txt)
    txt = ' '.join([word for word in txt.split()])
    return txt

In [15]:
def remplacement_de_mots(libelle) :
    ''' Fonction pour remplacer des mots dans un libelle '''

    # Dictionnaire des remplacements {'Mot à remplacer' : 'Mot de remplacement'}
    Dico_remplacements = {
                          'BOULEVARD' : ['BD'],
        'AVENUE' : ['AV'],
        'PLACE' : ['PL', 'PLAZA'],
        'ROUTE' : ['RT', 'RTE'],
        'SAINT' : ['ST', 'SAINTE'],
        'STE' : ['SOCIETE'],
        'ABATTOIR' : ['ABATTAGE', 'ABATTOIR', 'ABATTOIRS']

                         }

    # Traitement
    libelle_split = libelle.split()
    for mot_de_remplacement, Mots_a_remplacer in Dico_remplacements.items() :
        libelle_split = [mot if (mot not in Mots_a_remplacer) else mot_de_remplacement for mot in libelle_split]
        
    # Résultat
    return ' '.join(libelle_split)

In [16]:
from unidecode import unidecode
import re

def preprocess_text(texte) :
    
    # Suppression des espaces en préfixe/suffixe
    texte = texte.strip()
    
    # Mise en minuscules
    texte = texte.lower()
    
    # Suppression des espaces spéciaux
    texte = texte.replace(u'\xa0', u' ')
    
    # Suppression des caractères spéciaux
    texte = re.sub(r'\W',' ', texte)
    texte = texte.replace('_', ' ')
    
    # Suppression des caractères solos
    #texte = re.sub(r'\s+[a-zA-Z]\s+', ' ', texte)

    # Suppression des doubles espaces
    texte = re.sub(r'\s+', ' ', texte)
    
    
    
    # Résultat
    return unidecode(suppression_nombres(texte))

In [17]:
tqdm.pandas()
df['APP_Libelle_etablissement'] = df['APP_Libelle_etablissement'].astype(str).progress_apply(remplacement_de_mots)
test_df['APP_Libelle_etablissement'] = test_df['APP_Libelle_etablissement'].astype(str).progress_apply(remplacement_de_mots)

100%|██████████| 9717/9717 [00:00<00:00, 88903.54it/s]


In [18]:
tqdm.pandas()
df['APP_Libelle_etablissement'] = df['APP_Libelle_etablissement'].astype(str).progress_apply(preprocess_text)
test_df['APP_Libelle_etablissement'] = test_df['APP_Libelle_etablissement'].astype(str).progress_apply(preprocess_text)

100%|██████████| 9717/9717 [00:00<00:00, 60222.76it/s]


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tf-idf
tfidf_vec = TfidfVectorizer(max_features = 150,
                            stop_words = stopwords_list,
                           )

# Fit on train
tfidf_vec = tfidf_vec.fit(df['APP_Libelle_etablissement'].astype(str).values)

# =================================================================
# Apply on train
tfidf = tfidf_vec.transform(df['APP_Libelle_etablissement'].astype(str).values)
    
# Add tfidf features of df
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=['tfidf_' + x for x in tfidf_vec.get_feature_names_out()])
df = pd.concat([df, tfidf_df], axis=1)

# =================================================================
# Apply on train
tfidf = tfidf_vec.transform(test_df['APP_Libelle_etablissement'].astype(str).values)
    
# Add tfidf features of df
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=['tfidf_' + x for x in tfidf_vec.get_feature_names_out()])
test_df = pd.concat([test_df, tfidf_df], axis=1)

# Show
print(df.shape)
df.head(2)

(22671, 187)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok
0,saveurs asiatiques,50846842800034,RUE DES FRERES LUMIERE,33130,Bègles,23-110093-1,2024-02-05T01:00:00+01:00,Restaurant,Satisfaisant,NaN,44.797031_-0.535231,Restaurant,Autres,Restaurant,Bègles,SAVEURS ASIATIQUES,RUE DES FRERES LUMIERE,1,3,0,2.0,1,5,0,0,0,0,10,4536.0,4813.0,2011,9,2010,12,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,epicerie rond point,90472398800013,48 AV DU ROND POINT,93250,Villemomble,23-056603-1,2023-07-17T02:00:00+02:00,Libre service|Alimentation générale,Satisfaisant,NaN,48.884745_2.499984,Libre service|Alimentation générale,Autres,Li

In [20]:
tfidf_vec.get_feature_names_out()

array(['abattoir', 'accueil', 'agricole', 'andre', 'arc', 'ass',
       'association', 'atelier', 'auberge', 'auchan', 'bar', 'bat',
       'belle', 'bio', 'bistrot', 'bois', 'bon', 'boucherie',
       'boulangerie', 'brasserie', 'burger', 'cafe', 'cantine',
       'carrefour', 'centrale', 'centre', 'charcuterie', 'chateau',
       'chez', 'chicken', 'city', 'clinique', 'co', 'college', 'commune',
       'compagnie', 'comptoir', 'coop', 'cooperative', 'cote', 'creche',
       'creperie', 'cuisine', 'dame', 'delices', 'di', 'distribution',
       'domaine', 'donald', 'earl', 'ecole', 'ehpad', 'elementaire',
       'enfance', 'epicerie', 'esat', 'etablissements', 'ets', 'eurl',
       'exploitation', 'express', 'ferme', 'fils', 'fondation', 'food',
       'fournil', 'foyer', 'france', 'francois', 'franprix', 'freres',
       'fromagerie', 'gaec', 'gestion', 'gourmand', 'gourmet', 'grand',
       'grill', 'groupe', 'hopital', 'hospitalier', 'hotel', 'huitres',
       'intermarche', 'jardi

___
___
# **TFIDF on Adresse_2_UA**

In [21]:
tqdm.pandas()
df['Adresse_2_UA'] = df['Adresse_2_UA'].astype(str).progress_apply(remplacement_de_mots)
test_df['Adresse_2_UA'] = test_df['Adresse_2_UA'].astype(str).progress_apply(remplacement_de_mots)

100%|██████████| 9717/9717 [00:00<00:00, 81341.29it/s]


In [22]:
tqdm.pandas()
df['Adresse_2_UA'] = df['Adresse_2_UA'].astype(str).progress_apply(preprocess_text)
test_df['Adresse_2_UA'] = test_df['Adresse_2_UA'].astype(str).progress_apply(preprocess_text)

100%|██████████| 9717/9717 [00:00<00:00, 54021.74it/s]


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tf-idf
tfidf_vec = TfidfVectorizer(max_features = 15,
                            stop_words = stopwords_list,
                           )

# Fit on train
tfidf_vec = tfidf_vec.fit(df['Adresse_2_UA'].astype(str).values)

# =================================================================
# Apply on train
tfidf = tfidf_vec.transform(df['Adresse_2_UA'].astype(str).values)
    
# Add tfidf features of df
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=['tfidf_adresse_' + x for x in tfidf_vec.get_feature_names_out()])
df = pd.concat([df, tfidf_df], axis=1)

# =================================================================
# Apply on train
tfidf = tfidf_vec.transform(test_df['Adresse_2_UA'].astype(str).values)
    
# Add tfidf features of df
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=['tfidf_adresse_' + x for x in tfidf_vec.get_feature_names_out()])
test_df = pd.concat([test_df, tfidf_df], axis=1)

# Show
print(df.shape)
df.head(2)

(22671, 202)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone
0,saveurs asiatiques,50846842800034,rue des freres lumiere,33130,Bègles,23-110093-1,2024-02-05T01:00:00+01:00,Restaurant,Satisfaisant,NaN,44.797031_-0.535231,Restaurant,Autres,Restaurant,Bègles,SAVEURS ASIATIQUES,RUE DES FRERES LUMIERE,1,3,0,2.0,1,5,0,0,0,0,10,4536.0,4813.0,2011,9,2010,12,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [24]:
tfidf_vec.get_feature_names_out()

array(['avenue', 'boulevard', 'che', 'chemin', 'gaulle', 'general',
       'jean', 'nan', 'pierre', 'place', 'quai', 'route', 'rue', 'saint',
       'zone'], dtype=object)

___
___
# **Feature Engineering**

In [25]:
Villes = [
            "Paris",
            "Marseille",
            "Lyon",
            "Toulouse",
            "Nice",
            "Nantes",
            "Strasbourg",
            "Montpellier",
            "Bordeaux",
            "Lille",
            "Rennes",
            "Reims",
            "Le Havre",
            "Cannes",
            "Toulon",
            "Saint-Étienne",
            "Grenoble",
            "Dijon",
            "Angers",
            "Nîmes",
            "Nimes",
        ]

def process_str(x) :
    return str(x).lower().replace('é', 'e').replace('è', 'e')

Villes_processed = [process_str(x) for x in Villes]
Villes_processed

['paris',
 'marseille',
 'lyon',
 'toulouse',
 'nice',
 'nantes',
 'strasbourg',
 'montpellier',
 'bordeaux',
 'lille',
 'rennes',
 'reims',
 'le havre',
 'cannes',
 'toulon',
 'saint-etienne',
 'grenoble',
 'dijon',
 'angers',
 'nîmes',
 'nimes']

In [26]:
common_list = ['ECOLE PRIMAIRE PUBLIQUE',
                 'INTERMARCHE',
                 'CARREFOUR MARKET',
                 'ECOLE MATERNELLE',
                 'BAT PETITE ENFANCE',
                 'ECOLE ELEMENTAIRE',
                 'FRANPRIX',
                 'METRO FRANCE',
                 'BURGER KING',
                 'MONOPRIX',
                 'CARREFOUR CITY',
                 'CARREFOUR',
                 'ECOLE ELEMENTAIRE PUBLIQUE',
                 'CUISINE CENTRALE',
                 'AUCHAN SUPERMARCHE',
                 'ECOLE PRIMAIRE',
                 'SOCOPA VIANDES',
                 'PROMOCASH',
                 "MC DONALD'S",
                 'BUFFALO GRILL',
                 "MCDONALD'S",
                 'CDE',
                 'EHPAD',
                 'AUCHAN',
                 'GROUPE BIGARD',
                 'KFC',
                 'ALDI MARCHE',
                 'CARREFOUR CONTACT',
                 'CANTINE SCOLAIRE',
                 'DISTRIBUTION CASINO FRANCE',
                 'CORA',
                 'CHARAL',
                 'API RESTAURATION',
                 'LES PETITS CHAPERONS ROUGES',
                 'COOPERL ARC ATLANTIQUE',
                 'MAISON DE RETRAITE',
                 'ECOLE MATERNELLE PUBLIQUE',
                 'ELIOR RESTAURATION FRANCE',
                 'SUBWAY',
                 'SUPER U',
                 'BOULANGERIE MARIE BLACHERE',
                 'LIDL',
                 "DOMINO'S PIZZA",
                 "CENTRE COMMUNAL D'ACTION SOCIALE",
                 'SODEXO SANTE MEDICO SOCIAL',
                 "O'TACOS",
                 'COMPAGNIE DES FROMAGES ET RICHESMONTS',
                 'AU BUREAU',
                 'KERMENE',
                 'SODEXO SPORTS ET LOISIRS',
                 'ASS LES VIEILLES CHARRUES',
                 'TANG FRERES',
                 'ABATTOIR',
                 'CRECHE COLLECTIVE DEPARTEMENTALE',
                 'ENTREMONT ALLIANCE',
                 'S N V',
                 'GRAND FRAIS',
                 'CRESCENDO',
                 'E. LECLERC',
              ]

In [27]:
def find_ville(x, Villes_processed=Villes_processed) :
    x_processed = process_str(x)
    for ville in Villes_processed :
        if x_processed in ville or ville in x_processed :
            return ville
    return 'autre'
        
def FE(df, Villes=Villes, common_list=common_list, Activites=Activites) :
    
    # Ville et département
    df['Ville'] = df['Libelle_commune'].apply(find_ville)
    df['Departement'] = df['Code_postal'].apply(lambda x : str(x)[:2])
    
    # Flag Activités (0 ou 1)
    for act in Activites :
        df[f'flag_{act}'] = df['APP_Libelle_activite_etablissement'].astype(str).apply(lambda x : int(act in x)) + df['filtre'].astype(str).apply(lambda x : int(act in x))
        df[f'flag_{act}'] = (df[f'flag_{act}'] > 0).astype(int)
        
        
    # ============================================================================
    # Travail sur 'Agrement'
    mask = (~df['Agrement'].isna())
    df[f'Agrement_num_commune']    = df['Agrement'].astype(str).apply(lambda x : x[2:5] if len(x)>=8 else 999).astype(int)
    df[f'Agrement_num_entreprise'] = df['Agrement'].astype(str).apply(lambda x : x[-3:] if len(x)>=8 else 999).astype(int)
        
    # ============================================================================
    # Travail sur 'Numero_inspection'
    
    df['Numero_inspection_len']     = df['Numero_inspection'].astype(str).str.len()
    df['Numero_inspection_debut']   = df['Numero_inspection'].astype(str).apply(lambda x : int(x[:2])).astype(int)
    #df['Numero_inspection_milieu1'] = df['Numero_inspection'].astype(str).apply(lambda x : int(x[3:5])).astype(int)
    #df['Numero_inspection_milieu2'] = df['Numero_inspection'].astype(str).apply(lambda x : int(x[5:7])).astype(int)
    #df['Numero_inspection_milieu3'] = df['Numero_inspection'].astype(str).apply(lambda x : int(x[7:9])).astype(int)
    #df['Numero_inspection_milieu_all'] = df['Numero_inspection'].astype(str).apply(lambda x : int(x[3:9])).astype(int)
    df['Numero_inspection_fin']     = df['Numero_inspection'].astype(str).apply(lambda x : int(x[-1])).astype(int)
    
    #for i in [0, 1, 3, 4, 5, 6, 7, 8, 10] :
    #    df[f'Numero_inspection_caractere{i}'] = df['Numero_inspection'].astype(str).apply(lambda x : int(x[i]) if len(x)>i else -1).astype(int)
    
    # ============================================================================
    # Travail sur 'APP_Libelle_etablissement'
    
    df['is_common'] = df['APP_Libelle_etablissement'].astype(str).apply(lambda x : any(s in x for s in common_list)).astype(int)

    liste = [
                 'BURGER KING',
                 "MC DONALD'S",
                 'BUFFALO GRILL',
                 "MCDONALD'S",
                 'KFC',
                 'SUBWAY',
                 "DOMINO'S PIZZA",
                 "O'TACOS",
                 "MACDONALD", "MACDO",
              ]
    df['is_fastfood'] = df['APP_Libelle_etablissement'].astype(str).apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = [
                 'INTERMARCHE',
                 'CARREFOUR MARKET',
                 'FRANPRIX',
                 'MONOPRIX',
                 'CARREFOUR CITY',
                 'CARREFOUR',
                 'AUCHAN SUPERMARCHE',
                 'AUCHAN',
                 'ALDI MARCHE',
                 'CANTINE SCOLAIRE',
                 'DISTRIBUTION CASINO FRANCE',
                 'CORA',
                 'LIDL',
                 'GRAND FRAIS',
                 'E. LECLERC',
              ]
    df['is_grandesurface'] = df['APP_Libelle_etablissement'].astype(str).apply(lambda x : any(s in x for s in liste)).astype(int)
    
    liste =  ['ECOLE PRIMAIRE PUBLIQUE',
                 'ECOLE MATERNELLE',
                 'BAT PETITE ENFANCE',
                 'ECOLE ELEMENTAIRE',
                 'ECOLE ELEMENTAIRE PUBLIQUE',
                 'ECOLE PRIMAIRE',
                 'EHPAD',
                 'CANTINE SCOLAIRE',
                 'LES PETITS CHAPERONS ROUGES',
                 'MAISON DE RETRAITE',
                 'ECOLE MATERNELLE PUBLIQUE',
                 'ELIOR RESTAURATION FRANCE',
                 "CENTRE COMMUNAL D'ACTION SOCIALE",
                 'CRECHE COLLECTIVE DEPARTEMENTALE',
                 'ECOLE',
                 'CANTINE',
                 'SCOLAIRE',
              ]
    df['is_ecole_cantine'] = df['APP_Libelle_etablissement'].astype(str).apply(lambda x : any(s in x for s in liste)).astype(int)
            
    # ============================================================================
    # DATE (CV +0.06)
    df['Date_inspection'] = pd.to_datetime(df['Date_inspection'], utc=True)
    df['year']  = df['Date_inspection'].dt.year
    df['month'] = df['Date_inspection'].dt.month
    df['day']   = df['Date_inspection'].dt.day
    df['dayofweek']   = df['Date_inspection'].dt.dayofweek
    df['dayofyear']   = df['Date_inspection'].dt.dayofyear
    #df['hour']        = df['Date_inspection'].dt.hour # ALWAYS 0
        
    # ============================================================================
    # Travail sur 'FILTRE'
    
    df['filtre'] = df['filtre'].fillna('NAN')
    
    liste = ['Restaurant',
             'Restauration collective']
    df['is_restaurant'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = [
             'Boucherie-Charcuterie',
             'Boulangerie-Pâtisserie',
             'Traiteur',
             'Poissonnerie',
             'Fromagerie',
             'Glacier',
             'Chocolatier',
             'Primeur',
            ]
    df['is_vendeur'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = [
             'Rayon boucherie-charcuterie',
             'Rayon poissonnerie',
             'Rayon pain/viennoiserie/pâtisserie',
             'Rayon traiteur',
             'Rayon fromagerie',
            ]
    df['is_rayon'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = [
             'Fromagerie',
             'Rayon fromagerie',
            ]
    df['is_fromagerie'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = ['Boucherie-Charcuterie',
             'Rayon boucherie-charcuterie',
             ]
    df['is_boucherie'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = ['Poissonnerie',
             'Rayon poissonnerie',
            ]
    df['is_poissonnerie'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = ['Fromagerie',
             'Rayon fromagerie',
            ]
    df['is_fromagerie'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)

    liste = ['Alimentation générale',
             'Libre service',
            ]
    df['is_supermarche'] = df['filtre'].apply(lambda x : any(s in x for s in liste)).astype(int)
    
    # ============================================================================
    # Simplification du filtre
    
    df['len_filtre'] = df['filtre'].apply(lambda x : len(str(x).split('|')))
    mask = (df['len_filtre'] > 1 )
    df.loc[mask, 'filtre'] = df.loc[mask, 'filtre'].apply(lambda x : str(x).split('|')[0])
    
    # ============================================================================
    # Agrement manquant
    df['missing_agrement'] = (df['Agrement'].isna()).astype(int)
    
    # Return
    return df
    
df = FE(df)
test_df = FE(test_df)

print(df.shape)
df.head(2)

(22671, 296)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [28]:
print(test_df.shape)
test_df.head(2)

(9717, 296)


,trustii_id,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage 

___
___
# **Agrements**

In [29]:
# Scission des Agrements
def scission_agrement(test_df) :
    mask = (~test_df['Agrement'].isna())
    test_df.loc[mask, 'Agrement_1'] = test_df.loc[mask, 'Agrement'].apply(lambda x : str(x)[:8])
    mask = (~test_df['Agrement'].isna()) & (test_df['Agrement'].astype(str).str.len() > 10)
    test_df.loc[mask, 'Agrement_2'] = test_df.loc[mask, 'Agrement'].apply(lambda x : str(x)[9:])

    # Suppression des '.0' à la fin des agrements
    for k in ['Agrement', 'Agrement_1', 'Agrement_2'] :
        mask_ = (test_df[k].astype(str).str.endswith('.0'))
        test_df.loc[mask_, k] = test_df.loc[mask_, k].apply(lambda x : str(x)[:-2])
    return test_df

df      = scission_agrement(df)
test_df = scission_agrement(test_df)

In [30]:
mask = (~df['Agrement'].isna())

In [31]:
t = test_df.copy()

___
## **Number of controls per agrements**

In [32]:
row = df[df['Agrement_1'] == '02187002']
row

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [33]:
def complete_with_controls_same_day(df_to_complete, correct_biais = False) :
    
    #########################################################################
    # INITIALIZE COLUMNS
    df_to_complete[[f'other_y{v}_AGREMENTS_in_dftrain' for v in dico_y.values()]] = 0

    #########################################################################
    # PROCESS EACH ROW
    for i, row in tqdm(df_to_complete.iterrows(), total=len(df_to_complete)) :

        #########################################################################
        # FIND ROWS IN DF_TRAIN WHICH CORRESPOND TO THE SAME PLACE AND SAME DAY
        mask = None
        for cols in [['Agrement_1'],
                     ['Agrement_2'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Libelle_commune', 'Code_postal', 'filtre'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Libelle_commune', 'Code_postal'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Libelle_commune'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Code_postal'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Libelle_commune'],
                     ['APP_Libelle_etablissement', 'Adresse_2_UA', 'geores'],
                     ['APP_Libelle_etablissement', 'Adresse_2_UA', 'filtre'],
                     ['APP_Libelle_etablissement', 'geores'],
                     ['APP_Libelle_etablissement', 'filtre'],
                     ['APP_Libelle_etablissement', 'Adresse_2_UA'],
                     ['APP_Libelle_etablissement', 'Libelle_commune'],
                    ] :

            # MAJ du filtre
            m = None
            for k in ['Date_inspection'] + cols : # on ajoute la date d'inspection si on veut les controles du même jour
                if m is None :
                    m = (df[k] == row[k]).copy()
                else :
                    m = m & (df[k] == row[k])
            if mask is None :
                mask = m.copy()
            else :
                mask = mask | m

        # Gather all places
        same_place_same_day = df[mask].copy()

        #########################################################################
        # UPDATE DATA FOR THE ROW

        if len(same_place_same_day) > 0 :
            for k, v in dico_y.items() :
                n = len(same_place_same_day[same_place_same_day['Synthese_eval_sanit']==k])
                df_to_complete.at[i, f'other_y{v}_AGREMENTS_in_dftrain'] = n

    #########################################################################
    # ONCE ALL ROWS HAVE BEEN UPDATED ...
                
    # Add another column
    df_to_complete['N_agrements_in_train'] = df_to_complete[[f'other_y{v}_AGREMENTS_in_dftrain' for v in dico_y.values()]].sum(axis=1)

    # Correct biais (ne pas compter le controle en lui même dans la ligne)
    if correct_biais :
        m = (df_to_complete['N_agrements_in_train'] > 0)
        df_to_complete.loc[m, 'N_agrements_in_train'] = df_to_complete.loc[m, 'N_agrements_in_train'] - 1
        for k, v in dico_y.items() :
            col = f'other_y{v}_AGREMENTS_in_dftrain'
            m = (df_to_complete['Synthese_eval_sanit'] == k) & (df_to_complete[col] > 0)
            df_to_complete.loc[m, col] = df_to_complete.loc[m, col] - 1

    return df_to_complete

#df_to_complete = df[df['Agrement_1'] == '02187002'].copy()
#a = complete_with_controls_same_day(df_to_complete, correct_biais = True)
#a

In [34]:
%%time
df      = complete_with_controls_same_day(df, correct_biais = True)
test_df = complete_with_controls_same_day(test_df, correct_biais = False)

print(df.shape, test_df.shape)

100%|██████████| 9717/9717 [31:02<00:00,  5.22it/s]


(22671, 303) (9717, 303)
CPU times: user 1h 43min 51s, sys: 18.3 s, total: 1h 44min 10s
Wall time: 1h 44min 1s


In [35]:
cols = [f'other_y{v}_AGREMENTS_in_dftrain' for v in dico_y.values()]
display(df[cols].describe())
display(test_df[cols].describe())

,other_y0_AGREMENTS_in_dftrain,other_y1_AGREMENTS_in_dftrain,other_y2_AGREMENTS_in_dftrain,other_y3_AGREMENTS_in_dftrain
count,22671.000000,22671.000000,22671.000000,22671.000000
mean,0.000132,0.008866,0.091791,0.093291
std,0.011503,0.135661,0.397055,0.417826
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,1.000000,5.000000,5.000000,6.000000


,other_y0_AGREMENTS_in_dftrain,other_y1_AGREMENTS_in_dftrain,other_y2_AGREMENTS_in_dftrain,other_y3_AGREMENTS_in_dftrain
count,9717.000000,9717.000000,9717.000000,9717.000000
mean,0.000412,0.007307,0.096635,0.096017
std,0.020286,0.110428,0.421511,0.418467
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,1.000000,4.000000,6.000000,6.000000


In [36]:
#a = df.copy()
#a['n'] = df.groupby('Agrement_1')['SIRET'].transform('count')
#a.sort_values(by='n', ascending=False)

In [37]:
a = df[df['Agrement_1'] == '29233001'].sort_values(by = ['Date_inspection'])
a['Date_inspection'] = a['Date_inspection'].astype(str).apply(lambda x : x[:10])
a[['APP_Libelle_etablissement', 'Date_inspection', 'Synthese_eval_sanit']]

,APP_Libelle_etablissement,Date_inspection,Synthese_eval_sanit
20097,groupe bigard,2023-06-20,Satisfaisant
19536,groupe bigard,2023-07-07,Satisfaisant
17735,groupe bigard,2023-08-07,Très satisfaisant
13973,groupe bigard,2023-10-05,Satisfaisant
2706,groupe bigard,2023-11-09,Satisfaisant
21884,groupe bigard,2023-11-14,Très satisfaisant
12390,groupe bigard,2023-11-28,Satisfaisant


In [38]:
def complete_with_controls_other_days(df_to_complete) :

    #########################################################################
    # PROCESS EACH ROW
    for i, row in tqdm(df_to_complete.iterrows(), total=len(df_to_complete)) :

        #########################################################################
        # FIND ROWS IN DF_TRAIN WHICH CORRESPOND TO THE SAME PLACE AND SAME DAY
        mask = None
        for cols in [['Agrement_1'],
                     ['Agrement_2'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Libelle_commune', 'Code_postal', 'filtre'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Libelle_commune', 'Code_postal'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Libelle_commune'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Code_postal'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA'],
                     ['APP_Libelle_etablissement', 'SIRET', 'Libelle_commune'],
                     ['APP_Libelle_etablissement', 'Adresse_2_UA', 'geores'],
                     ['APP_Libelle_etablissement', 'Adresse_2_UA', 'filtre'],
                     ['APP_Libelle_etablissement', 'geores'],
                     ['APP_Libelle_etablissement', 'filtre'],
                     ['APP_Libelle_etablissement', 'Adresse_2_UA'],
                     ['APP_Libelle_etablissement', 'Libelle_commune'],
                    ] :

            # MAJ du filtre
            m = None
            for k in cols :
                if m is None :
                    m = (df[k] == row[k]).copy()
                else :
                    m = m & (df[k] == row[k])
            if mask is None :
                mask = m.copy()
            else :
                mask = mask | m

        # Gather all places
        mask = (df['Date_inspection'] != row['Date_inspection']) & mask # on ne veut pas les contrôles du même jour
        same_place_other_day = df[mask].copy()

        #########################################################################
        # UPDATE DATA FOR THE ROW
        
        if len(same_place_other_day) > 0 :
            for k, v in dico_y.items() :
                
                # All controls
                m = (same_place_other_day['Synthese_eval_sanit']==k)
                n = len(same_place_other_day[m])
                df_to_complete.at[i, f'other_y{v}_AGREMENTS_in_dftrain_other_day'] = n
                
                # Previous controls
                m = (same_place_other_day['Synthese_eval_sanit']==k) & (same_place_other_day['Date_inspection'] < row['Date_inspection'])
                n = len(same_place_other_day[m])
                df_to_complete.at[i, f'other_y{v}_AGREMENTS_in_dftrain_past'] = n
                
                # Future controls
                m = (same_place_other_day['Synthese_eval_sanit']==k) & (same_place_other_day['Date_inspection'] > row['Date_inspection'])
                n = len(same_place_other_day[m])
                df_to_complete.at[i, f'other_y{v}_AGREMENTS_in_dftrain_future'] = n
                
        # ----------------------------------------------------------------------
        # DAYS DIFF WITH OTHER CONTROLS
        
        # Calcul du nombre de jours entre les dates
        same_place_other_day['diff_days'] = same_place_other_day.apply(lambda r : (pd.to_datetime(r['Date_inspection']) - pd.to_datetime(row['Date_inspection'])).days, axis=1)

        # Previous controls
        m = (same_place_other_day['Date_inspection'] < row['Date_inspection'])
        controls = same_place_other_day[m].sort_values(by = ['Date_inspection'], ascending=False).copy()
        if len(controls)>0:
            min_ndays = controls['diff_days'].values[0]
            r1        = controls['Synthese_eval_sanit'].values[0]
            max_ndays = controls['diff_days'].values[-1]
            r2        = controls['Synthese_eval_sanit'].values[-1]
            df_to_complete.at[i, 'ndays_since_last_control']  = abs(min_ndays)
            df_to_complete.at[i, 'ndays_since_older_control'] = abs(max_ndays)
            df_to_complete.at[i, 'result_last_control']  = dico_y[r1]
            df_to_complete.at[i, 'result_older_control'] = dico_y[r2]
        if len(controls) >= 2 :
            df_to_complete.at[i, 'ndays_MEAN_since_past_controls'] = controls['diff_days'].mean()
            df_to_complete.at[i, 'ndays_STD_since_past_controls']  = controls['diff_days'].std()
            
        # Future controls
        m = (same_place_other_day['Date_inspection'] > row['Date_inspection'])
        controls = same_place_other_day[m].sort_values(by = ['Date_inspection'], ascending=True).copy()
        if len(controls)>0:
            min_ndays = controls['diff_days'].values[0]
            r1        = controls['Synthese_eval_sanit'].values[0]
            max_ndays = controls['diff_days'].values[-1]
            r2        = controls['Synthese_eval_sanit'].values[-1]
            df_to_complete.at[i, 'ndays_before_next_control']  = abs(min_ndays)
            df_to_complete.at[i, 'ndays_before_furthest_control'] = abs(max_ndays)
            df_to_complete.at[i, 'result_next_control']     = dico_y[r1]
            df_to_complete.at[i, 'result_furthest_control'] = dico_y[r2]
        if len(controls) >= 2 :
            df_to_complete.at[i, 'ndays_MEAN_before_futur_controls'] = controls['diff_days'].mean()
            df_to_complete.at[i, 'ndays_STD_before_futur_controls']  = controls['diff_days'].std()

    #########################################################################
    # ONCE ALL ROWS HAVE BEEN UPDATED ...
                
    # Add another column
    df_to_complete['N_agrements_in_train_other_day']  = df_to_complete[[f'other_y{v}_AGREMENTS_in_dftrain_other_day' for v in dico_y.values()]].sum(axis=1)
    df_to_complete['N_agrements_in_train_past']       = df_to_complete[[f'other_y{v}_AGREMENTS_in_dftrain_past' for v in dico_y.values()]].sum(axis=1)
    df_to_complete['N_agrements_in_train_future']     = df_to_complete[[f'other_y{v}_AGREMENTS_in_dftrain_future' for v in dico_y.values()]].sum(axis=1)

    # Fillna
    cols = ['ndays_since_last_control',
            'ndays_since_older_control',
            'ndays_before_next_control',
            'ndays_before_furthest_control',
            'ndays_MEAN_since_past_controls',
            'ndays_STD_since_past_controls',
            'ndays_MEAN_before_futur_controls',
            'ndays_STD_before_futur_controls',
           ]
    for k in cols :
        if k not in df_to_complete :
            df[k] = 9999
        else :
            df_to_complete[k] = df_to_complete[k].fillna(9999)
    
    cols = ['result_last_control',
            'result_older_control',
            'result_next_control',
            'result_furthest_control',
           ]
    for k in cols :
        if k not in df_to_complete :
            df[k] = -1
        else :
            df_to_complete[k] = df_to_complete[k].fillna(-1)
    
    # Return
    return df_to_complete

#df_to_complete = df[df['Agrement_1'] == '29233001'].sort_values(by = ['Date_inspection']).copy()
#complete_with_controls_other_days(df_to_complete)

In [39]:
%%time

df = complete_with_controls_other_days(df)
test_df = complete_with_controls_other_days(test_df)

print(df.shape, test_df.shape)
df.head()

100%|██████████| 9717/9717 [31:15<00:00,  5.18it/s]


(22671, 330) (9717, 330)
CPU times: user 1h 43min 50s, sys: 33.2 s, total: 1h 44min 24s
Wall time: 1h 44min 1s


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [40]:
df[['other_y0_AGREMENTS_in_dftrain',
    'other_y1_AGREMENTS_in_dftrain',
    'other_y2_AGREMENTS_in_dftrain',
    'other_y3_AGREMENTS_in_dftrain']].value_counts().to_frame()

count
other_y0_AGREMENTS_in_dftrain other_y1_AGREMENTS_in_dftrain other_y2_AGREMENTS_in_dftrain other_y3_AGREMENTS_in_dftrain       
0                             0                             0                             0                              19776
                                                                                          1                                944
                                                            1                             0                                891
                                                            2                             0                                292
                                                            0                             2                                236
                              1                             0                             0                                 93
                              0                             3                             0                                 79
                                                            0                             3                                 77
                                                            1                             1                                 59
                                                            2                             1                                 42
                                                            0                             4                                 36
                                                            4                             0                                 27
                                                            1                             2                                 20
                              2                             0                             0                                 19
                              0                             0                             5                                 18
                                                            3                             1                                  9
                              3                             0                             0                                  9
                              0                             1                             3                                  8
                                                            3                             5                                  6
                              5                             0                             0                                  6
                              0                             5                             0                                  6
                              1                             0                             1                                  4
                              2                             1                             0                                  3
                              0                             2                             6                                  3
1                             0                             0                             0                                  3
0                             1                             1                             0                                  2
                                                                                          2                                  1
                              0                             2                             2                                  1
                                                            1                             4                                  1

In [41]:
cols = ['N_agrements_in_train_other_day', 'N_agrements_in_train_past', 'N_agrements_in_train_future']
display(df[cols].describe())
display(test_df[cols].describe())

,N_agrements_in_train_other_day,N_agrements_in_train_past,N_agrements_in_train_future
count,22671.000000,22671.000000,22671.000000
mean,1.618191,0.809095,0.809095
std,8.117969,4.751355,4.748235
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,93.000000,93.000000,92.000000


,N_agrements_in_train_other_day,N_agrements_in_train_past,N_agrements_in_train_future
count,9717.000000,9717.000000,9717.000000
mean,1.673768,0.874241,0.799527
std,8.383030,5.237960,4.691441
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,94.000000,91.000000,92.000000


In [42]:
cols = ['N_agrements_in_train_other_day', 'N_agrements_in_train_past', 'N_agrements_in_train_future']
display(df[cols].describe())
display(test_df[cols].describe())

,N_agrements_in_train_other_day,N_agrements_in_train_past,N_agrements_in_train_future
count,22671.000000,22671.000000,22671.000000
mean,1.618191,0.809095,0.809095
std,8.117969,4.751355,4.748235
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,93.000000,93.000000,92.000000


,N_agrements_in_train_other_day,N_agrements_in_train_past,N_agrements_in_train_future
count,9717.000000,9717.000000,9717.000000
mean,1.673768,0.874241,0.799527
std,8.383030,5.237960,4.691441
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,94.000000,91.000000,92.000000


___
## **Number of controls BEFORE/AFTER per agrements**

In [43]:
def compute_bilan(df1, df2) :
    
    mask1 = (~df1['Agrement'].isna())
    mask2 = (~df2['Agrement'].isna())
    
    # Self-join
    cols = ['Agrement', 'Date_inspection', 'Synthese_eval_sanit']
    bilan_agrement = pd.merge(df1.loc[mask1, ['Agrement', 'Date_inspection']],
                              df2.loc[mask2, cols],
                              how = 'inner',
                              on = ['Agrement']
                             ).drop_duplicates()

    # Suppression des lignes identiques
    bilan_agrement = bilan_agrement[bilan_agrement['Date_inspection_x'] != bilan_agrement['Date_inspection_y']].reset_index(drop=True)

    # Calcul du nombre de jours entre les dates
    bilan_agrement['diff_days'] = bilan_agrement.apply(lambda row : (row['Date_inspection_x'] - row['Date_inspection_y']).days, axis=1)

    # Tri
    bilan_agrement.sort_values(by = ['Agrement', 'Date_inspection_x', 'diff_days'], ascending=True, inplace=True)
    bilan_agrement = bilan_agrement.reset_index(drop=True)

    # Encode
    bilan_agrement['Synthese_eval_sanit'] = bilan_agrement['Synthese_eval_sanit'].map(dico_y)

    # Df à compléter
    bilan_agrement_PASSE_FUTUR = df1.loc[mask1, ['Agrement', 'Date_inspection']].drop_duplicates().reset_index(drop=True)

    # ----------------------------------------------------------------------
    # Bilan PAST controls
    bilan_tmp = bilan_agrement[bilan_agrement['diff_days'] > 0].copy()
    bilan_tmp['rank'] = bilan_tmp.groupby(['Agrement', 'Date_inspection_x'])['diff_days'].rank(method='first').astype(int)

    # Add Data
    cols = ['Agrement', 'Date_inspection_x', 'Synthese_eval_sanit', 'diff_days']
    for rank in range(1, 5) :

        b = bilan_tmp[bilan_tmp['rank'] == rank][cols].rename(columns = {'Date_inspection_x' : "Date_inspection"})
        bilan_agrement_PASSE_FUTUR = pd.merge(bilan_agrement_PASSE_FUTUR,
                                              b.rename(columns = {'diff_days' : f"n_days_since_last_{rank}th_control",
                                                                  "Synthese_eval_sanit" : f"y_last_{rank}th_control",
                                                                 }),
                                              how = 'left',
                                              on = ['Agrement', 'Date_inspection']
                                             ).drop_duplicates()

        bilan_agrement_PASSE_FUTUR[f"n_days_since_last_{rank}th_control"] = bilan_agrement_PASSE_FUTUR[f"n_days_since_last_{rank}th_control"].fillna(9999)
        bilan_agrement_PASSE_FUTUR[f"y_last_{rank}th_control"] = bilan_agrement_PASSE_FUTUR[f"y_last_{rank}th_control"].fillna(-1)

    # ----------------------------------------------------------------------
    # Bilan FUTURE controls
    bilan_tmp = bilan_agrement[bilan_agrement['diff_days'] < 0].copy()
    bilan_tmp['diff_days'] = abs(bilan_tmp['diff_days'])
    bilan_tmp['rank'] = bilan_tmp.groupby(['Agrement', 'Date_inspection_x'])['diff_days'].rank(method='first', ascending=False).astype(int)

    # Add Data
    cols = ['Agrement', 'Date_inspection_x', 'Synthese_eval_sanit', 'diff_days']
    for rank in range(1, 5) :

        b = bilan_tmp[bilan_tmp['rank'] == rank][cols].rename(columns = {'Date_inspection_x' : "Date_inspection"})
        bilan_agrement_PASSE_FUTUR = pd.merge(bilan_agrement_PASSE_FUTUR,
                                              b.rename(columns = {'diff_days' : f"n_days_before_next_{rank}th_control",
                                                                  "Synthese_eval_sanit" : f"y_next_{rank}th_control",
                                                                 }),
                                              how = 'left',
                                              on = ['Agrement', 'Date_inspection']
                                             ).drop_duplicates()

        bilan_agrement_PASSE_FUTUR[f"n_days_before_next_{rank}th_control"] = bilan_agrement_PASSE_FUTUR[f"n_days_before_next_{rank}th_control"].fillna(9999)
        bilan_agrement_PASSE_FUTUR[f"y_next_{rank}th_control"] = bilan_agrement_PASSE_FUTUR[f"y_next_{rank}th_control"].fillna(-1)

    return bilan_agrement_PASSE_FUTUR

In [44]:
if not(debug) :
    bilan_agrement_PASSE_FUTUR = compute_bilan(df, df)

    # Aperçu
    display(bilan_agrement_PASSE_FUTUR[bilan_agrement_PASSE_FUTUR['Agrement'] == '01187001'])

,Agrement,Date_inspection,y_last_1th_control,n_days_since_last_1th_control,y_last_2th_control,n_days_since_last_2th_control,y_last_3th_control,n_days_since_last_3th_control,y_last_4th_control,n_days_since_last_4th_control,y_next_1th_control,n_days_before_next_1th_control,y_next_2th_control,n_days_before_next_2th_control,y_next_3th_control,n_days_before_next_3th_control,y_next_4th_control,n_days_before_next_4th_control
503,01187001,2024-01-17 00:00:00+00:00,2.0,49.0,3.0,330.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0
2272,01187001,2023-11-29 00:00:00+00:00,3.0,281.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0,3.0,49.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0
3451,01187001,2023-02-21 00:00:00+00:00,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0,-1.0,9999.0,3.0,330.0,2.0,281.0,-1.0,9999.0,-1.0,9999.0


In [45]:
if not(debug) :

    # Jointure
    df = pd.merge(df,
                  bilan_agrement_PASSE_FUTUR,
                  how = 'left',
                  on = ['Agrement', 'Date_inspection']
                 )

    # Fill NaN
    cols = [x for x in bilan_agrement_PASSE_FUTUR.columns if "y_" in x]
    df[cols] = df[cols].fillna(-1)
    cols = [x for x in bilan_agrement_PASSE_FUTUR.columns if "n_days" in x]
    df[cols] = df[cols].fillna(9999)

# Show
print(df.shape)
df.head(2)

(22671, 346)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [46]:
a = '02187002'
df[df['Agrement'] == a].sort_values(by = ['Date_inspection'])

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [47]:
a = '02187002'
test_df[test_df['Agrement'] == a].sort_values(by = ['Date_inspection'])

,trustii_id,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage 

In [48]:
if not(debug) :
    
    bilan_agrement_PASSE_FUTUR = compute_bilan(test_df, df)

    # Aperçu
    bilan_agrement_PASSE_FUTUR[bilan_agrement_PASSE_FUTUR['Agrement'] == a]

In [49]:
if not(debug) :
    
    # Jointure
    test_df = pd.merge(test_df,
                      bilan_agrement_PASSE_FUTUR,
                      how = 'left',
                      on = ['Agrement', 'Date_inspection']
                     )

    # Fill NaN
    cols = [x for x in bilan_agrement_PASSE_FUTUR.columns if "y_" in x]
    test_df[cols] = test_df[cols].fillna(-1)
    cols = [x for x in bilan_agrement_PASSE_FUTUR.columns if "n_days" in x]
    test_df[cols] = test_df[cols].fillna(9999)

# Show
print(test_df.shape)
test_df.head(2)

(9717, 346)


,trustii_id,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage 

In [50]:
print("Agrements feats OK !")

Agrements feats OK !


___
___
# **Geocoder : get missing latitudes/longitudes**

In [51]:
from geopy.geocoders import Nominatim

In [52]:
def get_coordinates(address):
    geolocator = Nominatim(user_agent="geo_locator")
    location = geolocator.geocode(address, timeout=30)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

In [53]:
def find_gps_coords(row) :
    combinaisons = [['Adresse_2_UA', 'Libelle_commune', 'Code_postal'],
                    ['APP_Libelle_etablissement', 'Libelle_commune', 'Code_postal'],
                    ['APP_Libelle_etablissement', 'Adresse_2_UA', 'Libelle_commune'],
                    ['APP_Libelle_etablissement', 'Adresse_2_UA'],
                    ['APP_Libelle_etablissement', 'Libelle_commune'],
                    ['APP_Libelle_etablissement', 'Code_postal'],
                    ['Libelle_commune', 'Code_postal'],
                   ]
    
    # Try different adresses
    try :
        for combi in combinaisons :
            adresse = ', '.join([str(row[x]) for x in combi])
            adresse = str(adresse).upper()

            lat, lon = get_coordinates(adresse)

            if lat is not None :
                return f"{lat}_{lon}"
    except :
        return ""
    
    return ""

def complete_gps_coords(df):

    mask_nan = (df['geores'].isna())
    print(f"Nombre de GEORES manquants INITIAL : {len(df[mask_nan])}")
    display(df[mask_nan].head(2))

    tqdm.pandas()
    df.loc[mask_nan, 'geores'] = df.loc[mask_nan].progress_apply(find_gps_coords, axis=1).astype(str)
    
    new_mask_nan = (df['geores'].isna()) | (df['geores'] == '')
    print(f"Nombre de GEORES manquants FINAL   : {len(df[new_mask_nan])}")
    display(df[mask_nan].head(2))
    
    return df

In [54]:
#mask_nan = (df['geores'].isna()) | (df['geores'] == '')
#a = df[mask_nan].reset_index(drop=True)
#complete_gps_coords(a.iloc[:10])
#find_gps_coords(a.iloc[0])

In [55]:
%%time

df = complete_gps_coords(df)

Nombre de GEORES manquants INITIAL : 607


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

100%|██████████| 607/607 [25:21<00:00,  2.51s/it]


Nombre de GEORES manquants FINAL   : 23


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

CPU times: user 48.1 s, sys: 2.65 s, total: 50.8 s
Wall time: 25min 22s


In [56]:
%%time

test_df = complete_gps_coords(test_df)

Nombre de GEORES manquants INITIAL : 257


,trustii_id,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage 

100%|██████████| 257/257 [11:06<00:00,  2.59s/it]


Nombre de GEORES manquants FINAL   : 4


,trustii_id,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage 

CPU times: user 21.7 s, sys: 1.15 s, total: 22.8 s
Wall time: 11min 7s


___
___
# **Concat train and test data to facilitate spatial information recovery**

In [57]:
test_df['Synthese_eval_sanit'] = 'NAN'
test_df['is_test'] = 1

df['is_test'] = 0

In [58]:
df = pd.concat([df, test_df]).reset_index(drop=True)

# Aperçu
print(df.shape)
df.head(2)

(32388, 348)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [59]:
df['is_test'].value_counts()

is_test
0    22671
1     9717
Name: count, dtype: int64

___
___
# **Handle still missing geores**

In [60]:
mask_geores = (~df['geores'].isna()) & (df['geores'].astype(str).str.len() > 1)
df.loc[mask_geores, 'latitude']  = df.loc[mask_geores, 'geores'].apply(lambda x : float(str(x).split('_')[0])).astype(float)
df.loc[mask_geores, 'longitude'] = df.loc[mask_geores, 'geores'].apply(lambda x : float(str(x).split('_')[1])).astype(float)
print(f"Nombre de geores trouvés : {len(df[mask_geores])}/{len(df)}")

Nombre de geores trouvés : 32361/32388


In [61]:
for i, row in tqdm(df[~mask_geores].iterrows(), total=len(df[~mask_geores])) :
    
    Libelle_commune = row['Libelle_commune']
    Code_postal     = row['Code_postal']
    Adresse_2_UA    = row['Adresse_2_UA']
    Departement     = row['Departement']
    
    found = False
    for LENGTH in [4, 3] :
        if found :
            break
        for mask in [(mask_geores) & (df['Libelle_commune']==Libelle_commune) & (df['Code_postal']==Code_postal),
                     (mask_geores) & (df['Libelle_commune']==Libelle_commune) & (df['Adresse_2_UA']==Adresse_2_UA),
                     (mask_geores) & (df['Code_postal']==Code_postal),
                     (mask_geores) & (df['Libelle_commune']==Libelle_commune) & (df['Departement']==Departement),
                    ]:
            if len(df[mask]) >= LENGTH :
                lat = df.loc[mask, 'latitude'].mean()
                lon = df.loc[mask, 'longitude'].mean()
                df.loc[i, 'geores'] = f"{str(lat)}_{str(lon)}"
                found = True
                break

100%|██████████| 27/27 [00:01<00:00, 18.57it/s]


In [62]:
mask_geores = (~df['geores'].isna()) & (df['geores'].astype(str).str.len() > 1)
df.loc[mask_geores, 'latitude']  = df.loc[mask_geores, 'geores'].apply(lambda x : float(str(x).split('_')[0])).astype(float)
df.loc[mask_geores, 'longitude'] = df.loc[mask_geores, 'geores'].apply(lambda x : float(str(x).split('_')[1])).astype(float)
print(f"Nombre de geores trouvés : {len(df[mask_geores])}/{len(df)}")

Nombre de geores trouvés : 32386/32388


___
___
# **Spatial information**

In [63]:
from scipy import spatial
def Compute_Mdist_Mindex(matrix, nb_max_candidates=1000, thr_distance=None) :
    
    nb_max_candidates = min(nb_max_candidates, len(matrix))
    
    latitudes  = matrix[:, 0] #* np.pi / 180
    longitudes = matrix[:, 1] #* np.pi / 180
    Z = tuple(zip(latitudes, longitudes)) # latitude, longitude

    tree = spatial.KDTree(Z)
    M_dist, M_index = (tree.query(Z, min(nb_max_candidates, len(matrix))))
    
    if thr_distance is None : 
        return M_dist, M_index

    # Threshold filter
    Nb_matches_potentiels = []
    for i in range(len(M_index)) :
        n = len([d for d in M_dist[i] if d <= thr_distance])
        Nb_matches_potentiels.append(n)
    M_dist  = [m[:Nb_matches_potentiels[i]] for i, m in enumerate(M_dist)]
    M_index = [m[:Nb_matches_potentiels[i]] for i, m in enumerate(M_index)]
    
    return M_dist, M_index

In [64]:
mask_geores = (~df['geores'].isna()) & (df['geores'].astype(str).str.len() > 1)
df.loc[mask_geores, 'latitude']  = df.loc[mask_geores, 'geores'].apply(lambda x : float(str(x).split('_')[0])).astype(float)
df.loc[mask_geores, 'longitude'] = df.loc[mask_geores, 'geores'].apply(lambda x : float(str(x).split('_')[1])).astype(float)
print(f"Nombre de geores trouvés : {len(df[mask_geores])}/{len(df)}")

Nombre de geores trouvés : 32386/32388


In [65]:
df[~mask_geores].head()

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

___
## **Find closest neighbours**

In [66]:
%%time

matrix = df.loc[mask_geores]
matrix['initial_id'] = matrix.index
matrix = matrix.reset_index(drop=True)

M_dist, M_index = Compute_Mdist_Mindex(matrix[['latitude', 'longitude']].to_numpy(),
                                       nb_max_candidates=4000,
                                       #thr_distance = 5,
                                      )

M_dist.shape

CPU times: user 19.5 s, sys: 833 ms, total: 20.3 s
Wall time: 20.3 s


(32386, 4000)

___
## **Compute infos about neighbours with the same activities**

In [67]:
INFOS = []
for idx1, (Liste_idx, Liste_val) in tqdm(enumerate(zip(M_index, M_dist)), total=len(matrix)):

    current_date_inspection = matrix.loc[idx1, 'Date_inspection']
    MAT = matrix.loc[Liste_idx[1:]]
    
    # Interdiction d'utiliser les données de test pour l'entrainement (en plus, on ne veut que les POI pour lesquels la variable 'Synthese_eval_sanit' est connue)
    MAT = MAT[MAT['is_test'] == 0]
    
    # Interdiction d'utiliser le même SIRET
    current_SIRET = matrix.loc[idx1, 'SIRET']
    MAT = MAT[MAT['SIRET'] != current_SIRET]
    
    # Debug
    """
    if idx1==501 :
        display(matrix.loc[[idx1]])
        print()
        print(Liste_val[1:6])
        print()
        print(MAT.shape)
        display(MAT.head())
        print()
    """
    
    # Initialisation des infos
    dict_info = {}
    
    # Distance aux commerces du même type d'activité
    for k, klib in [['APP_Libelle_activite_etablissement', 'LibAct'],
                    ['filtre', 'filtre'],
                    ['ods_type_activite', 'ods'],
                    ['Code_NAF_1', 'NAF1'],
                    ['activitePrincipaleEtablissement', 'actPrincipal'],
                    #['Synthese_eval_sanit', 'y'], # /!\ NON : créé des biais
                   ]:

        # Même type d'activité
        act = matrix.loc[idx1, k]
        mat = MAT[MAT[k] == act]
        mat['diff_days'] = mat['Date_inspection'].apply(lambda t : (current_date_inspection - t).days)

        # Distance du n-th plus proche commerce du même type d'activité
        for rank in range(10) :
            if rank < len(mat) :
                idx_to_find = mat.index[rank]
                dist = Liste_val[np.argwhere(Liste_idx == idx_to_find).flatten()][0]
                dict_info[f'dist_to_{rank+1}st_{klib}'] = dist
            else :
                dict_info[f'dist_to_{rank+1}st_{klib}'] = 9999

        # Moyenne et std des n-th plus proches voisins
        for rank in [3, 6, 10, 20] :
            if rank < len(mat) :
                idx_to_find = mat.index[:rank]
                dists = Liste_val[np.argwhere(np.isin(Liste_idx, idx_to_find)).flatten()]
                dict_info[f'MEAN_dist_to_{rank}th_nearest_{klib}'] = np.mean(dists)
                dict_info[f'STD_dist_to_{rank}th_nearest_{klib}']  = np.std(dists)
                dict_info[f'MAX_dist_to_{rank}th_nearest_{klib}']  = np.max(dists)

                # Nombre de contrôles (DATES D'INSPECTION)
                dict_info[f'Days_since_last_control_to_{rank}th_nearest_{klib}']    = np.min(mat.loc[mat['diff_days']>0, 'diff_days'])
                dict_info[f'Days_since_next_control_to_{rank}th_nearest_{klib}']    = np.min(mat.loc[mat['diff_days']<0, 'diff_days'])
                dict_info[f'Days_since_nearest_control_to_{rank}th_nearest_{klib}'] = np.min(abs(mat['diff_days']))

            else :
                dict_info[f'Days_since_last_control_to_{rank}_nearest_{klib}']    = 9999
                dict_info[f'Days_since_next_control_to_{rank}_nearest_{klib}']    = 9999
                dict_info[f'Days_since_nearest_control_to_{rank}_nearest_{klib}'] = 9999
                dict_info[f'MEAN_dist_to_{rank}th_nearest_{klib}'] = 9999
                dict_info[f'STD_dist_to_{rank}th_nearest_{klib}']  = 9999
                dict_info[f'MAX_dist_to_{rank}th_nearest_{klib}']  = 9999


    # APPEND
    INFOS.append(dict_info)

    # Debug
    #if idx1 > 15 : 
    #    break

# Aperçu des données calculées
df_spatial_1 = pd.DataFrame(INFOS).fillna(9999)

# Aperçu
print(df_spatial_1.shape)
df_spatial_1.head(2)

100%|██████████| 32386/32386 [55:10<00:00,  9.78it/s]


(32386, 230)


,dist_to_1st_LibAct,dist_to_2st_LibAct,dist_to_3st_LibAct,dist_to_4st_LibAct,dist_to_5st_LibAct,dist_to_6st_LibAct,dist_to_7st_LibAct,dist_to_8st_LibAct,dist_to_9st_LibAct,dist_to_10st_LibAct,MEAN_dist_to_3th_nearest_LibAct,STD_dist_to_3th_nearest_LibAct,MAX_dist_to_3th_nearest_LibAct,Days_since_last_control_to_3th_nearest_LibAct,Days_since_next_control_to_3th_nearest_LibAct,Days_since_nearest_control_to_3th_nearest_LibAct,MEAN_dist_to_6th_nearest_LibAct,STD_dist_to_6th_nearest_LibAct,MAX_dist_to_6th_nearest_LibAct,Days_since_last_control_to_6th_nearest_LibAct,Days_since_next_control_to_6th_nearest_LibAct,Days_since_nearest_control_to_6th_nearest_LibAct,MEAN_dist_to_10th_nearest_LibAct,STD_dist_to_10th_nearest_LibAct,MAX_dist_to_10th_nearest_LibAct,Days_since_last_control_to_10th_nearest_LibAct,Days_since_next_control_to_10th_nearest_LibAct,Days_since_nearest_control_to_10th_nearest_LibAct,MEAN_dist_to_20th_nearest_LibAct,STD_dist_to_20th_nearest_LibAct,MAX_dist_to_20th_nearest_LibAct,Days_since_last_control_to_20th_nearest_LibAct,Days_since_next_control_to_20th_nearest_LibAct,Days_since_nearest_control_to_20th_nearest_LibAct,dist_to_1st_filtre,dist_to_2st_filtre,dist_to_3st_filtre,dist_to_4st_filtre,dist_to_5st_filtre,dist_to_6st_filtre,dist_to_7st_filtre,dist_to_8st_filtre,dist_to_9st_filtre,dist_to_10st_filtre,MEAN_dist_to_3th_nearest_filtre,STD_dist_to_3th_nearest_filtre,MAX_dist_to_3th_nearest_filtre,Days_since_last_control_to_3th_nearest_filtre,Days_since_next_control_to_3th_nearest_filtre,Days_since_nearest_control_to_3th_nearest_filtre,MEAN_dist_to_6th_nearest_filtre,STD_dist_to_6th_nearest_filtre,MAX_dist_to_6th_nearest_filtre,Days_since_last_control_to_6th_nearest_filtre,Days_since_next_control_to_6th_nearest_filtre,Days_since_nearest_control_to_6th_nearest_filtre,MEAN_dist_to_10th_nearest_filtre,STD_dist_to_10th_nearest_filtre,MAX_dist_to_10th_nearest_filtre,Days_since_last_control_to_10th_nearest_filtre,Days_since_next_control_to_10th_nearest_filtre,Days_since_nearest_control_to_10th_nearest_filtre,MEAN_dist_to_20th_nearest_filtre,STD_dist_to_20th_nearest_filtre,MAX_dist_to_20th_nearest_filtre,Days_since_last_control_to_20th_nearest_filtre,Days_since_next_control_to_20th_nearest_filtre,Days_since_nearest_control_to_20th_nearest_filtre,dist_to_1st_ods,dist_to_2st_ods,dist_to_3st_ods,dist_to_4st_ods,dist_to_5st_ods,dist_to_6st_ods,dist_to_7st_ods,dist_to_8st_ods,dist_to_9st_ods,dist_to_10st_ods,MEAN_dist_to_3th_nearest_ods,STD_dist_to_3th_nearest_ods,MAX_dist_to_3th_nearest_ods,Days_since_last_control_to_3th_nearest_ods,Days_since_next_control_to_3th_nearest_ods,Days_since_nearest_control_to_3th_nearest_ods,MEAN_dist_to_6th_nearest_ods,STD_dist_to_6th_nearest_ods,MAX_dist_to_6th_nearest_ods,Days_since_last_control_to_6th_nearest_ods,Days_since_next_control_to_6th_nearest_ods,Days_since_nearest_control_to_6th_nearest_ods,MEAN_dist_to_10th_nearest_ods,STD_dist_to_10th_nearest_ods,MAX_dist_to_10th_nearest_ods,Days_since_last_control_to_10th_nearest_ods,Days_since_next_control_to_10th_nearest_ods,Days_since_nearest_control_to_10th_nearest_ods,MEAN_dist_to_20th_nearest_ods,STD_dist_to_20th_nearest_ods,MAX_dist_to_20th_nearest_ods,Days_since_last_control_to_20th_nearest_ods,Days_since_next_control_to_20th_nearest_ods,Days_since_nearest_control_to_20th_nearest_ods,dist_to_1st_NAF1,dist_to_2st_NAF1,dist_to_3st_NAF1,dist_to_4st_NAF1,dist_to_5st_NAF1,dist_to_6st_NAF1,dist_to_7st_NAF1,dist_to_8st_NAF1,dist_to_9st_NAF1,dist_to_10st_NAF1,MEAN_dist_to_3th_nearest_NAF1,STD_dist_to_3th_nearest_NAF1,MAX_dist_to_3th_nearest_NAF1,Days_since_last_control_to_3th_nearest_NAF1,Days_since_next_control_to_3th_nearest_NAF1,Days_since_nearest_control_to_3th_nearest_NAF1,MEAN_dist_to_6th_nearest_NAF1,STD_dist_to_6th_nearest_NAF1,MAX_dist_to_6th_nearest_NAF1,Days_since_last_control_to_6th_nearest_NAF1,Days_since_next_control_to_6th_nearest_NAF1,Days_since_nearest_control_to_6th_nearest_NAF1,MEAN_dist_to_10th_nearest_NAF1,STD_dist_to_10th_nearest

___
## **Compute infos about all neighbours**

In [68]:
INFOS = []
for idx1, (Liste_idx, Liste_val) in tqdm(enumerate(zip(M_index, M_dist)), total=len(matrix)):

    current_date_inspection = matrix.loc[idx1, 'Date_inspection']
    mat = matrix.loc[Liste_idx[1:]]
    
    # Interdiction d'utiliser les données de test pour l'entrainement (en plus, on ne veut que les POI pour lesquels la variable 'Synthese_eval_sanit' est connue)
    mat = mat[mat['is_test'] == 0]
    
    # Interdiction d'utiliser le même SIRET
    current_SIRET = matrix.loc[idx1, 'SIRET']
    mat = mat[mat['SIRET'] != current_SIRET]
    
    mat['diff_days'] = mat['Date_inspection'].apply(lambda t : (current_date_inspection - t).days)
    
    # Debug
    """
    if idx1==501 :
        display(matrix.loc[[idx1]])
        print()
        print(Liste_val[1:6])
        print()
        print(MAT.shape)
        display(MAT.head())
        print()
    """
    
    # Initialisation des infos
    dict_info = {}

    # Distance du n-th plus proche commerce du même type d'activité
    for rank in range(10) :
        if rank < len(mat) :
            idx_to_find = mat.index[rank]
            dist = Liste_val[np.argwhere(Liste_idx == idx_to_find).flatten()][0]
            dict_info[f'dist_to_REAL_{rank+1}st'] = dist
        else :
            dict_info[f'dist_to_REAL_{rank+1}st'] = 9999
            
    # Moyenne et std des n-th plus proches voisins
    for rank in [3, 6, 10, 15, 20, 50] :
        if rank < len(mat) :
            idx_to_find = mat.index[:rank]
            dists = Liste_val[np.argwhere(np.isin(Liste_idx, idx_to_find)).flatten()]
            dict_info[f'MEAN_dist_to_REAL_{rank}th_nearest'] = np.mean(dists)
            dict_info[f'STD_dist_to_REAL_{rank}th_nearest']  = np.std(dists)
            dict_info[f'MAX_dist_to_REAL_{rank}th_nearest']  = np.max(dists)

            # Nombre de contrôles (DATES D'INSPECTION)
            dict_info[f'Days_since_last_control_to_REAL_{rank}th_nearest']    = np.min(mat.loc[mat['diff_days']>0, 'diff_days'])
            dict_info[f'Days_since_next_control_to_REAL_{rank}th_nearest']    = np.min(mat.loc[mat['diff_days']<0, 'diff_days'])
            dict_info[f'Days_since_nearest_control_to_REAL_{rank}th_nearest'] = np.min(abs(mat['diff_days']))

        else :
            dict_info[f'Days_since_last_control_to_REAL_{rank}_nearest']    = 9999
            dict_info[f'Days_since_next_control_to_REAL_{rank}_nearest']    = 9999
            dict_info[f'Days_since_nearest_control_to_REAL_{rank}_nearest'] = 9999
            dict_info[f'MEAN_dist_to_REAL_{rank}th_nearest'] = 9999
            dict_info[f'STD_dist_to_REAL_{rank}th_nearest']  = 9999
            dict_info[f'MAX_dist_to_REAL_{rank}th_nearest']  = 9999
            
    # APPEND
    INFOS.append(dict_info)

    # Debug
    #if idx1 > 15 : 
    #    break
    
# Aperçu des données calculées
df_spatial_2a = pd.DataFrame(INFOS).fillna(9999)

# Aperçu
print(df_spatial_2a.shape)
df_spatial_2a.head(2)

100%|██████████| 32386/32386 [31:32<00:00, 17.12it/s]


(32386, 46)


,dist_to_REAL_1st,dist_to_REAL_2st,dist_to_REAL_3st,dist_to_REAL_4st,dist_to_REAL_5st,dist_to_REAL_6st,dist_to_REAL_7st,dist_to_REAL_8st,dist_to_REAL_9st,dist_to_REAL_10st,MEAN_dist_to_REAL_3th_nearest,STD_dist_to_REAL_3th_nearest,MAX_dist_to_REAL_3th_nearest,Days_since_last_control_to_REAL_3th_nearest,Days_since_next_control_to_REAL_3th_nearest,Days_since_nearest_control_to_REAL_3th_nearest,MEAN_dist_to_REAL_6th_nearest,STD_dist_to_REAL_6th_nearest,MAX_dist_to_REAL_6th_nearest,Days_since_last_control_to_REAL_6th_nearest,Days_since_next_control_to_REAL_6th_nearest,Days_since_nearest_control_to_REAL_6th_nearest,MEAN_dist_to_REAL_10th_nearest,STD_dist_to_REAL_10th_nearest,MAX_dist_to_REAL_10th_nearest,Days_since_last_control_to_REAL_10th_nearest,Days_since_next_control_to_REAL_10th_nearest,Days_since_nearest_control_to_REAL_10th_nearest,MEAN_dist_to_REAL_15th_nearest,STD_dist_to_REAL_15th_nearest,MAX_dist_to_REAL_15th_nearest,Days_since_last_control_to_REAL_15th_nearest,Days_since_next_control_to_REAL_15th_nearest,Days_since_nearest_control_to_REAL_15th_nearest,MEAN_dist_to_REAL_20th_nearest,STD_dist_to_REAL_20th_nearest,MAX_dist_to_REAL_20th_nearest,Days_since_last_control_to_REAL_20th_nearest,Days_since_next_control_to_REAL_20th_nearest,Days_since_nearest_control_to_REAL_20th_nearest,MEAN_dist_to_REAL_50th_nearest,STD_dist_to_REAL_50th_nearest,MAX_dist_to_REAL_50th_nearest,Days_since_last_control_to_REAL_50th_nearest,Days_since_next_control_to_REAL_50th_nearest,Days_since_nearest_control_to_REAL_50th_nearest
0,0.003236,0.004485,0.013662,0.017055,0.017055,0.020952,0.021490,0.022556,0.025197,0.032524,0.007128,0.004649,0.013662,3.0,-7.0,0,0.012741,0.006633,0.020952,3.0,-7.0,0,0.017821,0.008516,0.032524,3.0,-7.0,0,0.023724,0.010887,0.036438,3.0,-7.0,0,0.027915,0.011909,0.041920,3.0,-7.0,0,0.043848,0.015664,0.062494,3.0,-7.0,0
1,0.005843,0.007567,0.008685,0.010225,0.010653,0.010988,0.011007,0.011229,0.012644,0.012644,0.007365,0.001169,0.008685,4.0,-207.0,0,0.008994,0.001840,0.010988,4.0,-207.0,0,0.010149,0.002065,0.012644,4.0,-207.0,0,0.011499,0.002608,0.016109,4.0,-207.0,0,0.013006,0.003455,0.018008,4.0,-207.0,0,0.018806,0.005434,0.026419,4.0,-207.0,0


In [69]:
INFOS = []

for idx1, (Liste_idx, Liste_val) in tqdm(enumerate(zip(M_index, M_dist)), total=len(matrix)):

    dict_info = {}
    current_date_inspection = matrix.loc[idx1, 'Date_inspection']
    
    # Interdiction d'utiliser les données de test pour l'entrainement (en plus, on ne veut que les POI pour lesquels la variable 'Synthese_eval_sanit' est connue)
    MAT = matrix.loc[Liste_idx[1:]]
    MAT = MAT[MAT['is_test'] == 0]
    
    # Interdiction d'utiliser le même SIRET
    current_SIRET = matrix.loc[idx1, 'SIRET']
    MAT = MAT[MAT['SIRET'] != current_SIRET]
    
    # Distances des POI de df_train
    TRAIN_DISTS = Liste_val[np.argwhere(np.isin(Liste_idx, MAT.index)).flatten()]
    
    # Debug
    """
    if idx1==501 :
        display(matrix.loc[[idx1]])
        print()
        display(matrix.loc[Liste_idx[1:]].head())
        print()
        print(Liste_val[1:6])
        print(TRAIN_DISTS[:5])
        print()
        print(MAT.shape)
        display(MAT.head())
        print()
    """

    # Boucle sur les distances (rayon)
    for thr_distance in [0.03, 0.05, 0.075, 0.1, 0.15, 0.3, 0.6, 1, 1.5, 2, 4] :

        # Nombre de POI à proximité
        N = len([dist for dist in TRAIN_DISTS if dist <= thr_distance])
        dict_info[f"N_POI_within_{thr_distance}km"] = N

        # Distance moyenne
        n = np.mean(TRAIN_DISTS[:N])
        if pd.isna(n) :
            dict_info[f"MEAN_dist_POI_within_{thr_distance}km"] = 9999
        else :
            dict_info[f"MEAN_dist_POI_within_{thr_distance}km"] = n

        # Distance std
        n = np.std(TRAIN_DISTS[:N])
        if pd.isna(n) :
            dict_info[f"STD_dist_POI_within_{thr_distance}km"] = 9999
        else :
            dict_info[f"STD_dist_POI_within_{thr_distance}km"] = n

        # N plus proches voisins
        mat = MAT.loc[MAT.index[:N]]

        # Nombre de targets
        for target, target_num in dico_y.items() :
            dict_info[f"N_targets_{target_num}_within_{thr_distance}km"] = len(mat[mat['Synthese_eval_sanit'] == target])

        # Nombre de NAF1
        for t in range(7) :
            dict_info[f"N_NAF1_{t}_within_{thr_distance}km"] = len(mat[mat['Code_NAF_1'] == t])

        # Nombre de contrôles (DATES D'INSPECTION)
        N_DAYS = [7, 31, 61, 100]
        if len(mat)==0 :
            dict_info[f"N_controls_same_day_within_{thr_distance}km"] = 0
            for n_days in N_DAYS :
                dict_info[f"N_controls_within_{n_days}days_and_{thr_distance}km"] = 0
                dict_info[f"N_controls_within_{n_days}daysPOS_and_{thr_distance}km"] = 0
                dict_info[f"N_controls_within_{n_days}daysNEG_and_{thr_distance}km"] = 0
        else :
            mat['diff_days'] = mat['Date_inspection'].apply(lambda t : (current_date_inspection - t).days)
            dict_info[f"N_controls_same_day_within_{thr_distance}km"] = len(mat[mat['diff_days'] == 0])
            for n_days in N_DAYS :
                dict_info[f"N_controls_within_{n_days}days_and_{thr_distance}km"] = len(mat[mat['diff_days'].between(-n_days, n_days)])
                dict_info[f"N_controls_within_{n_days}daysPOS_and_{thr_distance}km"] = len(mat[mat['diff_days'].between(1, n_days)])
                dict_info[f"N_controls_within_{n_days}daysNEG_and_{thr_distance}km"] = len(mat[mat['diff_days'].between(-n_days, -1)])

    # APPEND
    INFOS.append(dict_info)

    # Debug
    #if idx1 > 5 : 
    #    break

# Aperçu des données calculées
df_spatial_2b = pd.DataFrame(INFOS).fillna(0)

# Aperçu
print(df_spatial_2b.shape)
df_spatial_2b.head(2)

100%|██████████| 32386/32386 [3:39:16<00:00,  2.46it/s]


(32386, 297)


,N_POI_within_0.03km,MEAN_dist_POI_within_0.03km,STD_dist_POI_within_0.03km,N_targets_0_within_0.03km,N_targets_1_within_0.03km,N_targets_2_within_0.03km,N_targets_3_within_0.03km,N_NAF1_0_within_0.03km,N_NAF1_1_within_0.03km,N_NAF1_2_within_0.03km,N_NAF1_3_within_0.03km,N_NAF1_4_within_0.03km,N_NAF1_5_within_0.03km,N_NAF1_6_within_0.03km,N_controls_same_day_within_0.03km,N_controls_within_7days_and_0.03km,N_controls_within_7daysPOS_and_0.03km,N_controls_within_7daysNEG_and_0.03km,N_controls_within_31days_and_0.03km,N_controls_within_31daysPOS_and_0.03km,N_controls_within_31daysNEG_and_0.03km,N_controls_within_61days_and_0.03km,N_controls_within_61daysPOS_and_0.03km,N_controls_within_61daysNEG_and_0.03km,N_controls_within_100days_and_0.03km,N_controls_within_100daysPOS_and_0.03km,N_controls_within_100daysNEG_and_0.03km,N_POI_within_0.05km,MEAN_dist_POI_within_0.05km,STD_dist_POI_within_0.05km,N_targets_0_within_0.05km,N_targets_1_within_0.05km,N_targets_2_within_0.05km,N_targets_3_within_0.05km,N_NAF1_0_within_0.05km,N_NAF1_1_within_0.05km,N_NAF1_2_within_0.05km,N_NAF1_3_within_0.05km,N_NAF1_4_within_0.05km,N_NAF1_5_within_0.05km,N_NAF1_6_within_0.05km,N_controls_same_day_within_0.05km,N_controls_within_7days_and_0.05km,N_controls_within_7daysPOS_and_0.05km,N_controls_within_7daysNEG_and_0.05km,N_controls_within_31days_and_0.05km,N_controls_within_31daysPOS_and_0.05km,N_controls_within_31daysNEG_and_0.05km,N_controls_within_61days_and_0.05km,N_controls_within_61daysPOS_and_0.05km,N_controls_within_61daysNEG_and_0.05km,N_controls_within_100days_and_0.05km,N_controls_within_100daysPOS_and_0.05km,N_controls_within_100daysNEG_and_0.05km,N_POI_within_0.075km,MEAN_dist_POI_within_0.075km,STD_dist_POI_within_0.075km,N_targets_0_within_0.075km,N_targets_1_within_0.075km,N_targets_2_within_0.075km,N_targets_3_within_0.075km,N_NAF1_0_within_0.075km,N_NAF1_1_within_0.075km,N_NAF1_2_within_0.075km,N_NAF1_3_within_0.075km,N_NAF1_4_within_0.075km,N_NAF1_5_within_0.075km,N_NAF1_6_within_0.075km,N_controls_same_day_within_0.075km,N_controls_within_7days_and_0.075km,N_controls_within_7daysPOS_and_0.075km,N_controls_within_7daysNEG_and_0.075km,N_controls_within_31days_and_0.075km,N_controls_within_31daysPOS_and_0.075km,N_controls_within_31daysNEG_and_0.075km,N_controls_within_61days_and_0.075km,N_controls_within_61daysPOS_and_0.075km,N_controls_within_61daysNEG_and_0.075km,N_controls_within_100days_and_0.075km,N_controls_within_100daysPOS_and_0.075km,N_controls_within_100daysNEG_and_0.075km,N_POI_within_0.1km,MEAN_dist_POI_within_0.1km,STD_dist_POI_within_0.1km,N_targets_0_within_0.1km,N_targets_1_within_0.1km,N_targets_2_within_0.1km,N_targets_3_within_0.1km,N_NAF1_0_within_0.1km,N_NAF1_1_within_0.1km,N_NAF1_2_within_0.1km,N_NAF1_3_within_0.1km,N_NAF1_4_within_0.1km,N_NAF1_5_within_0.1km,N_NAF1_6_within_0.1km,N_controls_same_day_within_0.1km,N_controls_within_7days_and_0.1km,N_controls_within_7daysPOS_and_0.1km,N_controls_within_7daysNEG_and_0.1km,N_controls_within_31days_and_0.1km,N_controls_within_31daysPOS_and_0.1km,N_controls_within_31daysNEG_and_0.1km,N_controls_within_61days_and_0.1km,N_controls_within_61daysPOS_and_0.1km,N_controls_within_61daysNEG_and_0.1km,N_controls_within_100days_and_0.1km,N_controls_within_100daysPOS_and_0.1km,N_controls_within_100daysNEG_and_0.1km,N_POI_within_0.15km,MEAN_dist_POI_within_0.15km,STD_dist_POI_within_0.15km,N_targets_0_within_0.15km,N_targets_1_within_0.15km,N_targets_2_within_0.15km,N_targets_3_within_0.15km,N_NAF1_0_within_0.15km,N_NAF1_1_within_0.15km,N_NAF1_2_within_0.15km,N_NAF1_3_within_0.15km,N_NAF1_4_within_0.15km,N_NAF1_5_within_0.15km,N_NAF1_6_within_0.15km,N_controls_same_day_within_0.15km,N_controls_within_7days_and_0.15km,N_controls_within_7daysPOS_and_0.15km,N_controls_within_7daysNEG_and_0.15km,N_controls_within_31days_and_0.15km,N_controls_within_31daysPOS_and_0.15km,N_controls_within_31daysNEG_and_0.15km,N_controls_within_61days_and_0.15km,N_controls_within_61daysPOS_and_0.15km,N_con

___
## **Add spatial info to df**

In [70]:
# Concat
df_spatial = pd.concat([df_spatial_2a, df_spatial_1, df_spatial_2b], axis=1)

# Set index
df_spatial.index = matrix['initial_id'].values

# Concat with df
df = pd.concat([df, df_spatial], axis=1)

# Fill NaN
cols_9999 = list(df_spatial_1.columns) + list(df_spatial_2a.columns) + [x for x in df_spatial_2b if any(s in x for s in ['MEAN_dist', 'STD_dist'])]
cols_0    = list(df_spatial_2b.columns)
df[cols_9999] = df[cols_9999].fillna(9999)
df[cols_0]    = df[cols_0].fillna(0)

___
___
# **Encoding**

In [71]:
def create_dico(k, df=df, encode_most_frequent_only=True) :
    N = len(df)
    dico = df[k].value_counts() / N
    if encode_most_frequent_only :
        dico = dico[dico > 0.01]
    dico = {k:i for i, k in enumerate(dico.index)}
    return dico, {v:k for k, v in dico.items()}

def encode_with_dict(df, col, dico) :
    return df[col].apply(lambda x : dico[x] if x in dico else len(dico))

In [72]:
dico_Libelle_activite, dico_Libelle_activite_INVERSE = create_dico('APP_Libelle_activite_etablissement')
dico_commune, dico_commune_INVERSE         = create_dico('Libelle_commune')
dico_ods, dico_ods_INVERSE                 = create_dico('ods_type_activite')
dico_filtre, dico_filtre_INVERSE           = create_dico('filtre')
dico_Ville, dico_Ville_INVERSE             = create_dico('Ville')
dico_Departement, dico_Departement_INVERSE = create_dico('Departement')

dico_Libelle_activite

{'Restaurant': 0,
 'Restauration collective': 1,
 'Producteur fermier': 2,
 'Transformation de lait ou produits laitiers': 3,
 'Boucherie-Charcuterie': 4,
 'Transformation de produits carnés': 5,
 "Chaîne d'abattage de volaille/lagomorphe/petit gibier": 6,
 'Boulangerie-Pâtisserie': 7,
 'Découpe de viande de boucherie': 8,
 'Purification/Expédition de coquillages': 9,
 'Libre service|Alimentation générale': 10,
 'Libre service': 11,
 "Centre d'emballage des oeufs": 12,
 'VH_ VSM_ préparation viandes boucherie': 13}

In [73]:
df['APP_Libelle_activite_etablissement'] = df['APP_Libelle_activite_etablissement'].apply(lambda x : dico_Libelle_activite[x] if x in dico_Libelle_activite else len(dico_Libelle_activite)).astype('category')
df['Libelle_commune']   = df['Libelle_commune'].apply(lambda x : dico_commune[x] if x in dico_commune else len(dico_commune)).astype('category')
df['ods_type_activite'] = df['ods_type_activite'].apply(lambda x : dico_ods[x] if x in dico_ods else len(dico_ods)).astype('category')
df['Ville']             = df['Ville'].apply(lambda x : dico_Ville[x] if x in dico_Ville else len(dico_Ville)).astype('category')
df['Departement']       = df['Departement'].apply(lambda x : dico_Departement[x] if x in dico_Departement else len(dico_Departement)).astype('category')
df['filtre']            = df['filtre'].apply(lambda x : dico_filtre[x] if x in dico_filtre else len(dico_filtre)).astype('category')

df.head()

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

___
___
# **Separate train and test**

In [74]:
test_df = df[df['is_test'] == 1].reset_index(drop=True).drop(columns = ['is_test'])
df      = df[df['is_test'] == 0].reset_index(drop=True).drop(columns = ['is_test'])
test_df['Synthese_eval_sanit'] = None

if 'trustii_id' in df :
    df = df.drop(columns = ['trustii_id'])

# Aperçu
print(df.shape, test_df.shape)
display(df.head())

(22671, 921) (9717, 922)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

## **Save result**

In [75]:
df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)